# Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# STEP 1: Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
# crypto_df.sample(10)
crypto_df.head()

## Explore the dataset

In [ ]:
# Checking the data types
crypto_df.dtypes

In [ ]:
# Checking the row counts
crypto_df.count()

In [ ]:
# Checking the column names
crypto_df.columns

In [ ]:
# Checking for duplicates
print(f"Duplicate entries: {crypto_df.duplicated().sum()}")


In [ ]:
# Renamed "Unnamed: 0" column and then set it as the index
crypto_df = crypto_df.rename(columns={'Unnamed: 0': ''})
crypto_df = crypto_df.set_index('')
# crypto_df.sample(10)
crypto_df.head()


# Transform the data set to be readable for machine learning

In [ ]:
# STEP 2: Keep all the cryptocurrencies that are being traded.
clean_crypto_df = crypto_df[crypto_df['IsTrading'] == True] 
clean_crypto_df

In [ ]:
# STEP 3: Keep all the cryptocurrencies that have a working algorithm.
clean_crypto_df = clean_crypto_df.dropna(subset=['Algorithm'])
clean_crypto_df

In [ ]:
# STEP 4: Drop the "IsTrading" column. 
del clean_crypto_df['IsTrading']
clean_crypto_df

In [ ]:
# Checking the row counts
clean_crypto_df.count()

In [ ]:
# STEP 5: Remove rows that have at least 1 null value.
clean_crypto_df = clean_crypto_df.dropna(how='any',axis=0) 
clean_crypto_df

In [ ]:
# Checking the row counts
clean_crypto_df.count()

In [ ]:
# STEP 6: Filter/Keep the rows where coins are mined.
clean_crypto_df = clean_crypto_df[clean_crypto_df['TotalCoinsMined'] > 0] 
clean_crypto_df

In [ ]:
# Checking the row counts
clean_crypto_df.count()

In [ ]:
# Step 7: Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = clean_crypto_df.filter(['CoinName'], axis=1)
crypto_names_df.head()

In [ ]:
# STEP 8: Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
del clean_crypto_df['CoinName']
clean_crypto_df.head()

In [ ]:
# STEP 9: Use get_dummies() to create variables for text features.
X = pd.get_dummies(clean_crypto_df, columns=["Algorithm", "ProofType"])
X

In [ ]:
# STEP 10: Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled)

## Deliverable 2: Reducing Data Dimensions Using Principal Component Analysis (PCA)

In [ ]:
# STEP 1: Clean and prepare the data, using the previous section.
# STEP 2: Using PCA to reduce dimension to three principal components. Initialize PCA model
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

In [ ]:
# STEP 3: Create a DataFrame with the three principal components: PC 1, PC2, PC 3.
pcs_df=pd.DataFrame(
data=X_pca, columns=["PC 1", "PC 2","PC 3"], index = clean_crypto_df.index)
pcs_df.head(10)

# Deliverable 3: Clustering Crytocurrencies Using K-Means

## Finding the Best Value for k Using the Elbow Curve

In [ ]:
# STEP 1: Use the previous section that converted the data to 3 dimensions using the Principal Component Analysis.
# STEP 2: Create an elbow curve to find the best value for K.
# Calculate the inertia for the range of K values
inertia = []
k = list(range(1, 11))
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

In [ ]:
# STEP 2: Create an elbow curve to find the best value for K.
# Graph the Elbow Curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)


In [ ]:
# STEP 3: Initialize the K-Means model, using k = 4 from reviewing the elbow curve.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

predictions


In [ ]:
# STEP 4: Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([clean_crypto_df, pcs_df], axis=1)

# STEP 5: Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_names_df 

# STEP 6: Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)


# Deliverable 4: Visualizing Cryptocurrencies Results

## 3D-Scatter with Clusters

In [ ]:
# STEP 1: Use the k-means method, picked k = 4
# STEP 2: Creating a 3D-Scatter with the PCA data and the clusters
# STEP 3: Add CoinName to hover_name, Algorithm to hover_data 
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=['Algorithm'],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [ ]:
# STEP 4: Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True, selectable=True)


In [ ]:
# STEP 5: Print the total number of tradable cryptocurrencies.
index = clustered_df.index
len(index)
print("There are " + str(len(index)) + " " + "tradable cryptocurrencies.")


In [ ]:
# STEP 6: Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
clustered_df_scaled = mms.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
clustered_df_scaled


In [ ]:
# STEP 7: Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=pd.DataFrame(
data=clustered_df_scaled, columns=['TotalCoinSupply','TotalCoinsMined'], index = clustered_df.index)

# STEP 8: Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName'] 


# STEP 9: Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class'] 

plot_df.head(10)


In [ ]:
# STEP 10: Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class", hover_cols=["CoinName"])
